In [1]:
import os
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
import pathlib 
import json
import imageio 
import tensorflow as tf 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow_hub as hub
from tqdm import tqdm
from moviepy.editor import  *
from collections import deque
%matplotlib inline

from sklearn.model_selection import train_test_split
# Run imports
from official.vision.configs import video_classification
from official.projects.movinet.configs import movinet as movinet_configs
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_layers
from official.projects.movinet.modeling import movinet_model
from official.projects.movinet.tools import export_saved_model
from tfrecords import parse_example
#from official.vision.beta.dataloaders import video_input



from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

In [ ]:
tf.test.gpu_device_name()
for gpu in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_virtual_device_configuration(gpu,[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4.5*1024)])
    

In [2]:
ss_files = pathlib.Path('.') # current path

datafile_path = pathlib.Path('./')/'records/tfrecords' #records folder
datafile_path.mkdir(parents=True, exist_ok=True)


datafile_prefix = pathlib.Path('dataset/data')  #path to videos 

# we will save the files in the same place as our datafiles ()
tfrecords_save_path = datafile_path
train_path = tfrecords_save_path/'train/' # path to training shards 
val_path = tfrecords_save_path/'val/' # path to val shards 
test_path = tfrecords_save_path/'test/' # path to test shards 


In [3]:
def get_shards(path):
    shards = [str(path) for path in path.iterdir() if '.tfrecord' == path.suffix]
    return shards

train_shards=get_shards(train_path)
val_shards=get_shards(val_path)
test_shards=get_shards(test_path)


In [4]:

# create dataset with train shards
# customize adding: .map(your_tf_function) after '.map(parse_example)\'
# Note: sharding allows us to shuffle the data efficiently
def get_data(shards):
  dataset = tf.data.TFRecordDataset(shards)
  

  dataset=dataset.map(parse_example)
  #dataset=dataset.repeat(1000)
  dataset=dataset.batch(1)
  data=iter(dataset)
  #data=dataset.as_numpy_iterator()
  return data


In [5]:
train_dataset=get_data(train_shards)
test_dataset=get_data(test_shards)
val_dataset=get_data(val_shards)
num_classes=50

In [ ]:
tf.compat.v1.data.get_output_types(train_dataset)

(tf.float32, tf.int64)

In [ ]:
tf.compat.v1.data.get_output_shapes(train_dataset)


(TensorShape([None, None, None, None, None]), TensorShape([None]))

In [ ]:
tf.compat.v1.data.get_output_classes(dataset)


(tensorflow.python.framework.ops.Tensor,
 tensorflow.python.framework.ops.Tensor,
 tensorflow.python.framework.ops.Tensor)

# OLD Model

In [ ]:
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=20, decay_rate=0.96, staircase=True
)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "signlanguage.h5", save_best_only=True
)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True
)
def create_model():
    model=Sequential()
    model.add(Input((None,224, 224, 3)))
    #model.add(Flatten())

    model.add(Conv3D(filters = 64, kernel_size = (1,3, 3), activation = 'relu'))
    model.add(Conv3D(filters = 64, kernel_size = (1,3, 3), activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size = (1,2, 2)))
    model.add(GlobalAveragePooling3D())
    model.add(Dense(256, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dense(2, activation = 'softmax'))

    model.summary()

    return model


In [ ]:
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=20, decay_rate=0.96, staircase=True
)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "signlanguage.h5", save_best_only=True
)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True
)
def get_model(width=224, height=224, depth=None):
    """Build a 3D convolutional neural network model."""

    inputs = Input((depth,height, width,  3))

    x = Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)
    #x=Flatten()(x)

    #x = GlobalAveragePooling3D()(x)
    #x = Dense(units=512, activation="relu")(x)
    #x = Dropout(0.3)(x)
    x=Bidirectional(ConvLSTM2D(512,kernel_size=3))(x)
    

    outputs = Dense(units=2, activation="sigmoid")(x)

    # Define the model.
    model = Model(inputs, outputs, name="3dcnn")
    model.summary()
    return model


# MoVi net


In [7]:

batch_size = 8
num_frames = 8
frame_stride = 10
resolution = 172


In [8]:
model_id = 'a0'

tf.keras.backend.clear_session()

backbone = movinet.Movinet(model_id=model_id)
model = movinet_model.MovinetClassifier(backbone=backbone, num_classes=600)
model.build([1, 1, 1, 1, 3])

# Load pretrained weights
!wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a0_base.tar.gz -O movinet_a0_base.tar.gz -q
!tar -xvf movinet_a0_base.tar.gz

checkpoint_dir = 'movinet_a0_base'
checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint = tf.train.Checkpoint(model=model)
status = checkpoint.restore(checkpoint_path)
status.assert_existing_objects_matched()

def build_classifier(backbone, num_classes, freeze_backbone=False):
  """Builds a classifier on top of a backbone model."""
  model = movinet_model.MovinetClassifier(
      backbone=backbone,
      num_classes=num_classes)
  model.build([batch_size, num_frames, resolution, resolution, 3])

  if freeze_backbone:
    for layer in model.layers[:-1]:
      layer.trainable = False
    model.layers[-1].trainable = True

  return model

# Wrap the backbone with a new classifier to create a new classifier head
# with num_classes outputs (101 classes for UCF101).
# Freeze all layers except for the final classifier head.
model = build_classifier(backbone, num_classes=num_classes, freeze_backbone=True)

x movinet_a0_base/
x movinet_a0_base/checkpoint
x movinet_a0_base/ckpt-1.data-00000-of-00001
x movinet_a0_base/ckpt-1.index


In [9]:
num_epochs = 3

train_steps = 720 // batch_size
total_train_steps = train_steps * num_epochs
test_steps = 450 // batch_size

loss_obj = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True,
    label_smoothing=0.1)

metrics = [
    tf.keras.metrics.TopKCategoricalAccuracy(
        k=1, name='top_1', dtype=tf.float32),
    tf.keras.metrics.TopKCategoricalAccuracy(
        k=5, name='top_5', dtype=tf.float32),
]

initial_learning_rate = 0.01
learning_rate = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate, decay_steps=total_train_steps,
)
optimizer = tf.keras.optimizers.RMSprop(
    learning_rate, rho=0.9, momentum=0.9, epsilon=1.0, clipnorm=1.0)

model.compile(loss=loss_obj, optimizer=optimizer, metrics=metrics)

callbacks = [
    tf.keras.callbacks.TensorBoard(),
]

In [10]:
results = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=1,
    steps_per_epoch=train_steps,
    validation_steps=test_steps,
    callbacks=callbacks,
    validation_freq=1,
    verbose=1)

ValueError: in user code:

    File "d:\final final code\env\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "d:\final final code\env\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\final final code\env\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "d:\final final code\env\lib\site-packages\keras\engine\training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "d:\final final code\env\lib\site-packages\keras\engine\training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "d:\final final code\env\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "d:\final

In [ ]:
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
d:\final final code\records train.ipynb Cell 18' in <cell line: 1>()
----> 1 results = model.fit(
      2     train_dataset,
      3     validation_data=test_dataset,
      4     epochs=1,
      5     steps_per_epoch=train_steps,
      6     validation_steps=test_steps,
      7     callbacks=callbacks,
      8     validation_freq=1,
      9     verbose=1)

File d:\final final code\env\lib\site-packages\keras\utils\traceback_utils.py:67, in filter_traceback.<locals>.error_handler(*args, **kwargs)
     65 except Exception as e:  # pylint: disable=broad-except
     66   filtered_tb = _process_traceback_frames(e.__traceback__)
---> 67   raise e.with_traceback(filtered_tb) from None
     68 finally:
     69   del filtered_tb

File d:\final final code\env\lib\site-packages\tensorflow\python\framework\func_graph.py:1147, in func_graph_from_py_func.<locals>.autograph_handler(*args, **kwargs)
   1145 except Exception as e:  # pylint:disable=broad-except
   1146   if hasattr(e, "ag_error_metadata"):
-> 1147     raise e.ag_error_metadata.to_exception(e)
   1148   else:
   1149     raise

ValueError: in user code:

    File "d:\final final code\env\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "d:\final final code\env\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\final final code\env\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "d:\final final code\env\lib\site-packages\keras\engine\training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "d:\final final code\env\lib\site-packages\keras\engine\training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "d:\final final code\env\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "d:\final final code\env\lib\site-packages\keras\losses.py", line 141, in __call__
        losses = call_fn(y_true, y_pred)
    File "d:\final final code\env\lib\site-packages\keras\losses.py", line 245, in call  **
        return ag_fn(y_true, y_pred, **self._fn_kwargs)
    File "d:\final final code\env\lib\site-packages\keras\losses.py", line 1789, in categorical_crossentropy
        return backend.categorical_crossentropy(
    File "d:\final final code\env\lib\site-packages\keras\backend.py", line 5083, in categorical_crossentropy
        target.shape.assert_is_compatible_with(output.shape)

    ValueError: Shapes (None, 1) and (None, 50) are incompatible
